导入库：
build_efficient_sam_vitt 和 build_efficient_sam_vits：用于构建 EfficientSAM 模型。
Image：用于图像处理。
transforms：用于将图像转换为张量。
torch：PyTorch 库。
np：NumPy 库。
zipfile：用于解压缩权重文件。
加载模型：
创建一个字典 models，其中包含一个 EfficientSAM 模型 (efficientsam_ti)。
从压缩文件中提取权重文件。

In [1]:
from efficient_sam.build_efficient_sam import build_efficient_sam_vitt, build_efficient_sam_vits
from PIL import Image
from torchvision import transforms
import torch
import numpy as np
import zipfile
models = {}
models['efficientsam_ti'] = build_efficient_sam_vitt()
with zipfile.ZipFile("weights/efficient_sam_vits.pt.zip", 'r') as zip_ref:
    zip_ref.extractall("weights")
sample_image_np = np.array(Image.open("figs/examples/dog1.jpg"))
sample_image_tensor = transforms.ToTensor()(sample_image_np)

d:\anaconda\envs\EFF_SAM\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


inference函数
1. 首先遍历模型对每个模型进行推理
2. 然后将图像张量、输入点和标签传入模型、获取预测的掩码和IOU
3. 更具IOU分数排序，选择最高的掩码
4. 将掩码应用到图像上并保存结果

In [3]:
def inference():
    for model_name, model in models.items():
        print('Running inference using ', model_name)
        predicted_logits, predicted_iou = model(
            sample_image_tensor[None, ...],
            input_points.to(torch.float32),
            input_labels.to(torch.float32),
        )
        sorted_ids = torch.argsort(predicted_iou, dim=-1, descending=True)
        predicted_iou = torch.take_along_dim(predicted_iou, sorted_ids, dim=2)
        predicted_logits = torch.take_along_dim(
            predicted_logits, sorted_ids[..., None, None], dim=2
        )
        # The masks are already sorted by their predicted IOUs.
        # The first dimension is the batch size (we have a single image. so it is 1).
        # The second dimension is the number of masks we want to generate (in this case, it is only 1)
        # The third dimension is the number of candidate masks output by the model.
        # For this demo we use the first mask.
        mask = torch.ge(predicted_logits[0, 0, 0, :, :], 0).cpu().detach().numpy()
        masked_image_np = sample_image_np.copy().astype(np.uint8) * mask[:,:,None]
        Image.fromarray(masked_image_np).save(f"figs/examples/dog1_{model_name}_mask_ui.png")

on_EVENT_LBUTTONDOWN函数
处理鼠标左键点击，当左键点击时，记录点击的坐标，在图像上绘制一个红点并显示点击的坐标
 

In [4]:
def on_EVENT_LBUTTONDOWN(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        xy = "%d,%d" % (x, y)
        a.append(x)
        b.append(y)
        cv2.circle(img, (x, y), 1, (0, 0, 255), thickness=-1)
        cv2.putText(img, xy, (x, y), cv2.FONT_HERSHEY_PLAIN,
                    1.0, (0, 0, 0), thickness=1)
        cv2.imshow("image", img)
 
count = 0

主循环初始化：

使用 OpenCV 加载和显示图像，等待用户点击。
记录点击位置并生成输入点和标签。
调用 inference 函数对图像进行掩码生成并保存结果。
第一次运行：

读取初始图像并显示，等待用户点击两次以记录点坐标。
生成输入点和标签，调用 inference 进行处理并保存结果图像。
增加 count，避免重复初始化。
后续运行：

读取处理后的图像，等待用户点击记录新点。
根据新点击位置生成输入点和标签，调用 inference 进行处理。

In [5]:
for i in range(1):
    import cv2
    import numpy as np
    # 图片路径
    model_name = "efficientsam_ti"
    a = []
    b = []
    if count == 0:
        img = cv2.imread(f"figs/examples/dog1.jpg")
        cv2.imshow("image", img)
        cv2.setMouseCallback("image", on_EVENT_LBUTTONDOWN)
        # cv2.imshow("image", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        positions = [[[[a[0], b[0]], [a[1], b[1]]]]]
        input_points = torch.tensor(positions)
        sample_image_np = np.array(img)
        sample_image_tensor = transforms.ToTensor()(sample_image_np)
        input_labels = torch.tensor([[[1, 1]]])
        inference()
        count += 1
        continue
    a = []
    b = []
    img = cv2.imread(f"figs/examples/dog1_{model_name}_mask.png")
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", on_EVENT_LBUTTONDOWN)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    positions = [[[[a[0],b[0]],[a[1],b[1]]]]]
    input_points = torch.tensor(positions)
    sample_image_np = np.array(img)
    sample_image_tensor = transforms.ToTensor()(sample_image_np)
    input_labels = torch.tensor([[[1, 1]]])
    print("---------------------------------------------")
    for i in range(len(a)):
        print(a[i], b[i])
    inference()

Running inference using  efficientsam_ti
